# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from KineticAssembly_AD import ReactionNetwork, VectorizedRxnNetExp, VecSim, Optimizer, EquilibriumSolver,OptimizerExp
import networkx as nx
import torch
from torch import DoubleTensor as Tensor
import numpy as np

We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../../input_files/ap2_dG_trap.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()


FileNotFoundError: [Errno 2] No such file or directory: '../../input_files/ap2_dG_trap.pwr'

In [ ]:
print(rn.dG_is_param)

In [ ]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

In [ ]:
#Do modifications here
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([2]*np.array(1e0))
# new_kon = new_kon +torch.rand(len(new_kon))*4

#Random initial condition
#Latest
# new_kon = new_kon + Tensor([1.0784531126282055, 1.029458105931473, 1.1269561454873085, 1.1016504619786531, 1.1962540960808816, 12.733516534843671, 13.127248934123227, 1.0685841067754172, 12.268238406610628, 11.377777346624722, 3.567418136298749, 5.936616399850786, 13.945665881026535, 13.370150974891361, 11.895765670366242, 4.970355439491927, 11.479711161807957, 2.391746953140913, 13.204855105632266, 11.736181219089554, 4.7102150063254795, 7.89012439713287])


# new_kon = new_kon + Tensor([0.5742, 0.5904, 0.5740, 0.5455, 0.5514, 7.5604, 7.9476, 0.5958, 7.0857,
#         6.1911, 5.7815, 4.7904, 8.7658, 8.1895, 6.7626, 3.8649, 6.3190, 1.3492,
#         8.0216, 6.5788, 3.7168, 6.6780])

# new_kon = new_kon + Tensor([3.875147930751502, 3.633157460430131, 3.07085193445574, 0.9518820841788759, 1.0494634790517816, 3.875147930751502, 3.875147930751502, 1.0627547697426531, 3.633157460430131, 3.633157460430131, 1.0627547697426531, 3.633157460430131, 3.07085193445574, 3.07085193445574, 1.0627547697426531, 3.07085193445574, 3.875147930751502, 3.875147930751502, 3.633157460430131, 1.0494634790517816, 3.875147930751502, 3.875147930751502])


#Starting condition for HomoRates
# rates = [0.2498580000880769, 1.5458277554618909, 1.3286185272052629]
# rates=[5,5,5]
# rates=[0.8469241906100582, 2.0549138907600244, 7.962836081464261]
# rates=[0.6910176821168917, 1.893485325644041, 8.187479054602493]

# rates = [0.7638575677886703, 1.2275762212163108, 1.2292556662570815] # Current
# rates = [0.3667160530555285, 4.056637284043357, 4.197374196441568]
# rates= [0.6090738370299691, 2.0618771938391056, 8.36260499321765]
# rates = [0.31993640267233014, 3.439671796780782, 3.669072000528544]
# rates = [0.7754529778993657, 2.7661023329487895, 5.638537667297305]
# rates=[0.1668, 2.8192, 2.8560]
rates=[0.29646059227830807, 3.8221716863515653, 3.5559205371359406]
counter=0
for k,v in rn.rxn_class.items():
    for rid in v:
        new_kon[v] = rates[counter]
    counter+=1



update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
vec_rn = VectorizedRxnNet_Exp(rn, dev='cpu')
print(vec_rn.kon)

#Changing initial concentrations

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [ ]:
print(rn.dG_is_param)
print(vec_rn.dG_is_param)
# print(rn.ddG_fluc)

## Using the optimizer with a 1 second simulation runtime ##

In [ ]:
# learn_rate=[1e-3,1e-3]
learn_rate=1e-3
momentum=0.2
runtime=10

files_range=[100,500,1000,5000,10000]
yield_thresh=0.8
yield_min=0.7

In [ ]:

vec_rn.reset(reset_params=True)
optim = OptimizerExp(reaction_network=vec_rn,
                  sim_runtime=runtime,
                  optim_iterations=100,
                  learning_rate=learn_rate,
                  device='cpu',method="Adam",reg_penalty=1000000,mom=momentum)


## Conc based optimization


In [ ]:
# files_range=[50,80,100,120,150]
# optim.rn.update_reaction_net(rn)
# optim.optimize_wrt_conc(conc_scale=1e-1,conc_thresh=1e-1,mod_bool=True,mod_factor=10,max_thresh=1e2,max_yield=0,yield_species=14,conc_files_pref="ConcProfile_Time_HomoRates_",conc_files_range=files_range,time_threshmax=1)

### TIme based optimization

In [ ]:
# optim.rn.update_reaction_net(rn)
# optim.optimize_wrt_time(conc_scale=1e-1,conc_thresh=1e-1,mod_bool=True,mod_factor=10,max_thresh=1e2,max_yield=0,yield_species=14,
#                         conc_file="ConcProfile_Time_1000.0uM",time_threshmin=1e-3,time_threshmax=2e-3)

In [ ]:
optim.optimizer.load_state_dict(torch.load("Opt_state_01"))

In [ ]:
# files_range=[10,50,100,500,1000,5000]
# files_range=[10,50,100,1000]

optim.rn.update_reaction_net(rn)
optim.optimize_wrt_conc_beta(conc_scale=1e-1,conc_thresh=1e-1,mod_bool=True,mod_factor=10,max_thresh=1e2,max_yield=0,yield_species=14,
                        conc_files_pref="dG_trap/ConcProfile_Time_HomoRates_",conc_files_range=files_range,yield_threshmin=yield_min,yield_threshmax=yield_thresh)

In [ ]:
path_state = "Opt_state_01"

print(optim.optimizer.state_dict())
torch.save(optim.optimizer.state_dict(),path_state)

In [ ]:
# # files_range=[10,50,100,500,1000,5000]
# # files_range=[10,50,100,1000]

# # optim.rn.update_reaction_net(rn)
# optim.optimize_wrt_conc_beta(conc_scale=1e-1,conc_thresh=1e-1,mod_bool=True,mod_factor=10,max_thresh=1e2,max_yield=0,yield_species=14,
#                         conc_files_pref="dG_param/ConcProfile_Time_HomoRates_",conc_files_range=files_range,yield_threshmin=yield_min,yield_threshmax=yield_thresh)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
fig,ax = plt.subplots()

ax.plot(optim.mse_error)

ax.tick_params(labelsize='xx-large')

ax.set_xlabel("Iterations",fontsize=25)
ax.set_ylabel("MSE",fontsize=25)


# ax.legend(fontsize='large')

In [ ]:
print(optim.parameter_history[-1])

In [ ]:
yields= []
final_params=[]
mse_error = []
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i].item())
#     print(optim.final_solns[i].numpy())
    final_params.append(optim.final_solns[i].numpy())
    mse_error.append(optim.mse_error[i])
    
sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)#[sort_indx]
sorted_params = np.array(final_params)#[sort_indx]
mse_error = np.array(mse_error)



print(list(sorted_params[-1]))


In [ ]:
print(len(optim.mse_error))

In [ ]:
yields= []
final_params=[]
mse_error = []

for i in range(len(optim.yield_per_iter)):
    yields.append(optim.yield_per_iter[i])
    params=[]
    for j in range(len(optim.parameter_history[i])):
        params.append(np.array(optim.parameter_history[i][j]))
    final_params.append(params)
    mse_error.append(optim.mse_error[i])
    
sorted_yields=np.array(yields)#[sort_indx]
sorted_params = np.array(final_params)#[sort_indx]
mse_error = np.array(mse_error)

print(sorted_params[-1])

In [ ]:
min_indx = np.argmin(mse_error)

min_rates = list(sorted_params[min_indx])
min_error = mse_error[min_indx]

# dG = -1*torch.log(min_rates[0][0]*vec_rn._C0/min_rates[1][0])
print("Params: ",min_rates)
# print("dG: ",dG)
print("Min SSE: ",min_error )

## Writing when koff is also a parameter

In [ ]:
# #Writing all solutions to a file

# klabels=['k'+str(i) for i in range(len(vec_rn.kon))]
# header = '#Yield\t' + "\t".join(klabels) + "\tt50\tt85\tt95\n"

# files_range = [str(f) for f in files_range]
# filestr = ",".join(files_range)


# with open("Solutions_Conc_Homorates_dGparam_07_part2",'a') as fl:
#     fl.write(header)
#     fl.write("# Range of Concentrations: %s\n" %filestr)
#     fl.write("# Learning rate: %s\n" %(",".join(str(lr) for lr in learn_rate)))
#     fl.write("# Momentum: %f\n" %(momentum))
#     fl.write("# Yield thresh: %f\n" %(yield_thresh))
#     for i in range(len(sorted_yields)):
#         fl.write("%f" %(sorted_yields[i]))
#         fl.write("\t%f" %(mse_error[i]))
#         for j in range((sorted_params[i].shape[0])):
#             for k in range(len(sorted_params[i][j])):
#                 fl.write("\t%f" %(sorted_params[i][j][k]))
#         fl.write("\n")
        
                 


### Writing outpout when only kon is parameter

In [ ]:
#Writing all solutions to a file

klabels=['k'+str(i) for i in range(len(vec_rn.kon))]
header = '#Yield\t' + "\t".join(klabels) + "\tt50\tt85\tt95\n"

files_range = [str(f) for f in files_range]
filestr = ",".join(files_range)


with open("Solutions_Conc_Homorates_dGNotrap_02_part1",'a') as fl:
    fl.write(header)
    fl.write("# Range of Concentrations: %s\n" %filestr)
    fl.write("# Learning rate: %s\n" %(str(learn_rate)))
    fl.write("# Momentum: %f\n" %(momentum))
    fl.write("# Yield thresh: %f\n" %(yield_thresh))
    for i in range(len(sorted_yields)):
        fl.write("%f" %(sorted_yields[i]))
        fl.write("\t%f" %(mse_error[i]))
        for j in range((sorted_params[i].shape[0])):
            fl.write("\t%f" %(sorted_params[i][j]))
        fl.write("\n")
                 


In [ ]:
fig,ax = plt.subplots()

k1=[]
for i in range(len(sorted_params)):
    k1.append(sorted_params[i][0])
ax.plot(k1)

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','S','M','AB','BMS','ABS','AMS','ABMS','AM','AS']
#nodes_list = ['A','B','ABMS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:
optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i],)
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
plt.show()

In [ ]:
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 

In [ ]:
import torch.nn as nn

loss = nn.MSELoss()

bconc=Tensor([10,12,14,20,25])
b.requires_grad=True
a = Tensor([3,5,6,7,8,9,10])
a.requires_grad=True
b = Tensor([4,5,6,7,8])
b.requires_grad=True

c = a.expand(b.shape[0], a.shape[0])
c2 = b.expand(a.shape[0], b.shape[0])

d = torch.abs(c-c2.T)

print(d.shape)

indx = d.argmin(dim=1)

e = a[indx]
print(e)
f = 2*e

cost=loss(f,bconc)
print(cost)

cost.backward()
print(b)

In [ ]:
a = Tensor([0])
a.requires_grad=True
b = Tensor([10])
b.requires_grad=True

for i in range(10):
    a=torch.cat((a,b))
    
print(a)
